In [43]:
from pandas import Series,DataFrame
from numpy import *
import numpy as np
import pandas as pd
import pandas
import codecs
import torch 
import torch.nn as nn 
from torch.autograd import Variable 
import torch.utils.data as Data 
import torchvision 
import matplotlib.pyplot as plt 
import csv
from sklearn.model_selection import cross_val_score,train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.svm import SVR 
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import explained_variance_score
from sklearn import metrics
from sklearn.metrics import mean_absolute_error # 平方绝对误差
import random


In [44]:
df = pd.read_excel(r'sel_all _1.xlsx',header =0)

In [45]:
feature = df[['LipoaffinityIndex','MLogP','maxHsOH','minsssN','MDEC-23','nC','maxssO',
             'minHsOH','BCUTc-1l','maxsssN','MLFER_A','minssO','SHsOH',
              'SsOH','maxsOH','ATSp5','SwHBa','C1SP2','nHBAcc','minHBint5']]
#feature = df[['C1SP2','nHBAcc','ATSc2','MDEC-23','C3SP2','nHBAcc_Lipinski','ATSc1',
#              'ETA_BetaP_s','BCUTc-1h']]



In [46]:
pIC50 = df.values[:, -1]
pIC50 = pd.DataFrame(pIC50)

In [5]:
class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(20, 1000),
            torch.nn.Dropout(0.2),
            torch.nn.ReLU(),
            torch.nn.Linear(1000, 800),
            torch.nn.Dropout(0.2),
            torch.nn.ReLU(),
            torch.nn.Linear(800, 600),
            torch.nn.Dropout(0.2),
            torch.nn.ReLU(),
            torch.nn.Linear(600, 500),
            torch.nn.Dropout(0.2),
            torch.nn.ReLU(),
            torch.nn.Linear(500, 300),
            torch.nn.Dropout(0.2),
            torch.nn.ReLU(),
            torch.nn.Linear(300, 250),
            torch.nn.Dropout(0.2),
            torch.nn.ReLU(),
            torch.nn.Linear(250, 1)
        )

        # init

    def forward(self, x):
        x = self.mlp(x)
        return x
#model_dict=torch.load("ANN.pth")

In [24]:
feature = feature.astype(float)
feature = torch.from_numpy(feature.values)
feature = feature.float()
pIC50 = pIC50.astype(float)
pIC50 = torch.from_numpy(pIC50.values)
pIC50 = pIC50.float()


In [25]:
lossData=[[]]
model = ANN()

In [15]:
loss_func = nn.MSELoss()
optim = torch.optim.Adam(model.parameters(), 0.0001)
batch_size =64
epochs = 200
print("epoch\t loss\t")
losses = []
for i in range(epochs):
     batch_loss = []
    # start和end分别是提取一个batch数据的起始和终止下标
     for start in range(0, len(feature), batch_size):
        end = start + batch_size if start + batch_size < len(feature) else len(feature)
        xx = torch.tensor(feature[start:end], dtype = torch.float, requires_grad = False)
        yy = torch.tensor(pIC50[start:end], dtype = torch.float, requires_grad = False)
        predict = model(xx)
        loss =loss_func(predict, yy)
        optim.zero_grad()
        loss.backward()
        optim.step()
        batch_loss.append(loss.data.numpy())
    
    # 每隔100步输出一下损失值（loss）
     if i % 10==0:
        losses.append(np.mean(batch_loss))
        print(i, np.mean(batch_loss))

epoch	 loss	
0

<ipython-input-15-db6a3aa0ddb1>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xx = torch.tensor(feature[start:end], dtype = torch.float, requires_grad = False)
<ipython-input-15-db6a3aa0ddb1>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  yy = torch.tensor(pIC50[start:end], dtype = torch.float, requires_grad = False)


 1.5836023
10 1.1053729
20 1.1023707
30 1.1978526
40 1.094581
50 1.1963027
60 1.2005458
70 1.1519866
80 1.1269196
90 1.1101768
100 1.0952128
110 1.2230247
120 1.0663292
130 1.0883863
140 1.2265183
150 1.0828196
160 1.1023023
170 1.102582
180 1.081222
190 1.0940244


In [17]:
torch.save(model,"ANN1.pth")

In [71]:
class PSO:
    def __init__(self, parameters):
        """
        particle swarm optimization
        parameter: a list type, like [NGEN, pop_size, var_num_min, var_num_max]
        """
        # 初始化
        self.NGEN = parameters[0]    # 迭代的代数
        self.pop_size = parameters[1]    # 种群大小
        self.var_num = len(parameters[2])     # 变量个数
        self.bound = []                 # 变量的约束范围
        self.bound.append(parameters[2])
        self.bound.append(parameters[3])
 
        self.pop_x = np.zeros((self.pop_size, self.var_num))    # 所有粒子的位置
        self.pop_v = np.zeros((self.pop_size, self.var_num))    # 所有粒子的速度
        self.p_best = np.zeros((self.pop_size, self.var_num))   # 每个粒子最优的位置
        self.g_best = np.zeros((1, self.var_num))   # 全局最优的位置
 
        # 初始化第0代初始全局最优解
        temp = -1
        for i in range(self.pop_size):
            for j in range(self.var_num):
                self.pop_x[i][j] = random.uniform(self.bound[0][j], self.bound[1][j])
                self.pop_v[i][j] = random.uniform(0, 1)
            self.p_best[i] = self.pop_x[i]      # 储存最优的个体
            fit = self.fitness(self.p_best[i])
            if fit > temp:
                self.g_best = self.p_best[i]
                temp = fit
 
    def fitness(self, ind_var):
        X = feature_train
        y = target_train
        """
        个体适应值计算
        """
        x1 = ind_var[0]
        x2 = ind_var[1]
        x3 = ind_var[2]
        
        if x1==0:x1=0.001
        if x2==0:x2=0.001
        if x3==0:x3=0.001
            
        clf = SVR(C=x1,epsilon=x2,gamma=x3)
        clf.fit(X, y)
        
        predictval=Func(feature_test)
        
        predictval1=predictval[0:int(len(predictval))]
        
        print("R2 = ",metrics.r2_score(target_test,predictval1)) # R2
        
        return  metrics.r2_score(target_test,predictval1)
 
    def update_operator(self, pop_size):
        """
        更新算子：更新下一时刻的位置和速度
        """
        c1 = 2     # 学习因子，一般为2
        c2 = 2
        w = 0.4    # 自身权重因子
        for i in range(pop_size):
            # 更新速度
            self.pop_v[i] = w * self.pop_v[i] + c1 * random.uniform(0, 1) * (
                        self.p_best[i] - self.pop_x[i]) + c2 * random.uniform(0, 1) * (self.g_best - self.pop_x[i])
            # 更新位置
            self.pop_x[i] = self.pop_x[i] + self.pop_v[i]
            # 越界保护
            for j in range(self.var_num):
                if self.pop_x[i][j] < self.bound[0][j]:
                    self.pop_x[i][j] = self.bound[0][j]
                if self.pop_x[i][j] > self.bound[1][j]:
                    self.pop_x[i][j] = self.bound[1][j]
            # 更新p_best和g_best
            if self.fitness(self.pop_x[i]) > self.fitness(self.p_best[i]):
                self.p_best[i] = self.pop_x[i]
            if self.fitness(self.pop_x[i]) > self.fitness(self.g_best):
                self.g_best = self.pop_x[i]
 
    def main(self):
        popobj = []
        self.ng_best = np.zeros((1, self.var_num))[0]
        for gen in range(self.NGEN):
            self.update_operator(self.pop_size)
            popobj.append(self.fitness(self.g_best))
            print('############ Generation {} ############'.format(str(gen + 1)))
            if self.fitness(self.g_best) > self.fitness(self.ng_best):
                self.ng_best = self.g_best.copy()
            print('最好的位置：{}'.format(self.ng_best))
            print('最大的函数值：{}'.format(self.fitness(self.ng_best)))
        print("---- End of (successful) Searching ----")
 
        plt.figure()
        fig = plt.gcf()
        fig.set_size_inches(18.5, 10.5)
        plt.title("Figure1")
        plt.xlabel("iterators", size=14)
        plt.ylabel("fitness", size=14)
        t = [t for t in range(self.NGEN)]
        plt.plot(t, popobj, color='b', linewidth=2)
        plt.show()


In [63]:
model_dict_ANN=torch.load("ANN1.pth")

In [49]:
def Func(a):
    a = np.array(a)
    a= torch.tensor(a)
    a = a.to(torch.float32)
    j = -model_dict_ANN(a)
    k = j.detach().numpy()
    return k


In [50]:
a = Func(feature.values[1,])

In [51]:
variable_range=pd.read_excel(r'第四题取值范围.xlsx').values#  可变参数的取值范围
lb = variable_range[0,1:]
ub = variable_range[1,1:]
lb=np.array(lb,dtype=np.float)
ub=np.array(ub,dtype=np.float)


In [59]:
feature_train,feature_test,target_train,target_test = train_test_split(feature,pIC50,test_size=0.1,random_state=8)


In [61]:
feature_test.shape

(54, 20)

In [72]:

if __name__ == '__main__':
    NGEN = 20
    popsize = 20
    low = lb
    up = ub
    parameters = [NGEN, popsize, low, up]
    pso = PSO(parameters)
    pso.main()


IndexError: list index out of range

In [54]:
print(xopt , fopt)

[ 6.61088022e+00  3.13483547e+00  8.45795817e-01  2.90563041e-02
  3.63222191e+01  2.29470590e+01  1.55842584e+00  1.35166661e-01
 -2.88584434e-01  1.53167211e+00  1.08038021e+00  6.18007706e-01
  2.30553964e+00  3.97200000e+01  3.24311274e+00  5.25000000e+02
  4.05276790e+01  4.85985999e+00  8.12711228e-02  1.04593882e+01] [-12.203024]
